In [1]:
import keras
keras.__version__
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

Using TensorFlow backend.
c:\anaconda3\envs\tensorflow1\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\anaconda3\envs\tensorflow1\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\anaconda3\envs\tensorflow1\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\anaconda3\envs\tensorflow1\lib\site-packages\tensorflow\python

In [2]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

In [3]:
print (train_data.shape, test_data.shape)
print (train_targets.shape, test_targets.shape)

(404, 13) (102, 13)
(404,) (102,)


- train 데이터는 404개 test 데이터는 102개
- 1개 데이터는 13 feature가 있고 보스턴 집값관련 특성이다.
1. Per capita crime rate.
2. Proportion of residential land zoned for lots over 25,000 square feet.
3. Proportion of non-retail business acres per town.
4. Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
5. Nitric oxides concentration (parts per 10 million).
6. Average number of rooms per dwelling.
7. Proportion of owner-occupied units built prior to 1940.
8. Weighted distances to five Boston employment centres.
9. Index of accessibility to radial highways.
10. Full-value property-tax rate per $10,000.
11. Pupil-teacher ratio by town.
12. 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town.
13. % lower status of the population.

In [4]:
print(np.round(train_data[1], 1), np.round(train_targets[1], 3))

[  0.   82.5   2.    0.    0.4   7.6  15.7   6.3   2.  348.   14.7 395.4
   3.1] 42.3


### Data Normalization

In [5]:
def MeanStdScaler(train_data, test_data):
    mean = train_data.mean(axis=0)
    train_data -= mean
    std = train_data.std(axis=0)
    train_data /= std

    test_data -= mean
    test_data /= std

    # print(mean, std)
    train_data.shape[1]
    return train_data, test_data

In [6]:
MeanStdScaler(train_data, test_data)

(array([[-0.27224633, -0.48361547, -0.43576161, ...,  1.14850044,
          0.44807713,  0.8252202 ],
        [-0.40342651,  2.99178419, -1.33391162, ..., -1.71818909,
          0.43190599, -1.32920239],
        [ 0.1249402 , -0.48361547,  1.0283258 , ...,  0.78447637,
          0.22061726, -1.30850006],
        ...,
        [-0.40202987,  0.99079651, -0.7415148 , ..., -0.71712291,
          0.07943894, -0.67776904],
        [-0.17292018, -0.48361547,  1.24588095, ..., -1.71818909,
         -0.98764362,  0.42083466],
        [-0.40422614,  2.04394792, -1.20161456, ..., -1.30866202,
          0.23317118, -1.15392266]]),
 array([[ 1.55369355, -0.48361547,  1.0283258 , ...,  0.78447637,
         -3.48459553,  2.25092074],
        [-0.39242675, -0.48361547, -0.16087773, ..., -0.30759583,
          0.42733126,  0.47880119],
        [-0.39982927, -0.48361547, -0.86940196, ...,  0.78447637,
          0.44807713, -0.41415936],
        ...,
        [-0.20709507, -0.48361547,  1.24588095, ..., -

### Model Creation

In [7]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    #model.summary()
    
    return model

- Regression일 때 마지막 레이어에 Softmax라던지 sigmoid 등의 activation function이 없다.
- Classification일 때는 분류를 해야 하므로 softmax 또는 Sigmoid 활용

### K-flod validation

In [8]:
import numpy as np

k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []
for i in range(k):
    print('처리중인 폴드는 #', i)
    
    # 검증 데이터 준비: k번째 분할
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    print('shape', val_data.shape, val_targets.shape)
    
    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate([train_data[:i * num_val_samples], train_data[(i + 1) * num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples], train_targets[(i + 1) * num_val_samples:]], axis=0)
    print('shape', partial_train_data.shape, partial_train_targets.shape)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=10, verbose=1)
    
    #검증 세트로 모델 평가
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

처리중인 폴드는 # 0
shape (101, 13) (101,)
shape (303, 13) (303,)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
303/303 [==============================] - 0s 1ms/step - loss: 516.0872 - mae: 21.0339
Epoch 2/100
303/303 [==============================] - 0s 297us/step - loss: 360.0110 - mae: 17.2663
Epoch 3/100
303/303 [==============================] - 0s 289us/step - loss: 204.7479 - mae: 12.3415
Epoch 4/100
303/303 [==============================] - 0s 274us/step - loss: 93.0092 - mae: 7.5129
Epoch 5/100
303/303 [==============================] - 0s 285us/step - loss: 45.7140 - mae: 5.0175
Epoch 6/100
303/303 [==============================] - 0s 279us/step - loss: 33.0973 - mae: 4.1117
Epoch 7/100
303/303 [==============================] - 0s 267us/step - loss: 27.4470 - mae: 3.6946
Epoch 8/100
303/303 [==============================] - 0s 287us/step - loss: 24.3793 - mae: 3.4330
Epoch 9/100
303/303 [====

303/303 [==============================] - 0s 277us/step - loss: 6.6600 - mae: 1.7852
Epoch 77/100
303/303 [==============================] - 0s 271us/step - loss: 6.4582 - mae: 1.7734
Epoch 78/100
303/303 [==============================] - 0s 265us/step - loss: 6.7667 - mae: 1.8150
Epoch 79/100
303/303 [==============================] - 0s 221us/step - loss: 6.4237 - mae: 1.7464
Epoch 80/100
303/303 [==============================] - 0s 258us/step - loss: 6.7004 - mae: 1.7877
Epoch 81/100
303/303 [==============================] - 0s 272us/step - loss: 6.5232 - mae: 1.7508
Epoch 82/100
303/303 [==============================] - 0s 259us/step - loss: 6.5720 - mae: 1.7795
Epoch 83/100
303/303 [==============================] - 0s 270us/step - loss: 6.5126 - mae: 1.7293
Epoch 84/100
303/303 [==============================] - 0s 250us/step - loss: 6.2952 - mae: 1.7279
Epoch 85/100
303/303 [==============================] - 0s 268us/step - loss: 6.4385 - mae: 1.7241
Epoch 86/100
303/303 [=

303/303 [==============================] - 0s 240us/step - loss: 7.6177 - mae: 1.9060
Epoch 59/100
303/303 [==============================] - 0s 245us/step - loss: 7.3369 - mae: 1.8645
Epoch 60/100
303/303 [==============================] - 0s 289us/step - loss: 7.7468 - mae: 1.9192
Epoch 61/100
303/303 [==============================] - 0s 276us/step - loss: 7.5344 - mae: 1.9076
Epoch 62/100
303/303 [==============================] - 0s 268us/step - loss: 7.4590 - mae: 1.8862
Epoch 63/100
303/303 [==============================] - 0s 257us/step - loss: 7.4910 - mae: 1.8693
Epoch 64/100
303/303 [==============================] - 0s 252us/step - loss: 7.5087 - mae: 1.9149
Epoch 65/100
303/303 [==============================] - 0s 265us/step - loss: 7.3221 - mae: 1.8627
Epoch 66/100
303/303 [==============================] - 0s 263us/step - loss: 7.5223 - mae: 1.8782
Epoch 67/100
303/303 [==============================] - 0s 290us/step - loss: 7.0941 - mae: 1.8545
Epoch 68/100
303/303 [=

303/303 [==============================] - 0s 275us/step - loss: 7.3459 - mae: 1.9832
Epoch 41/100
303/303 [==============================] - 0s 275us/step - loss: 7.2051 - mae: 1.9668
Epoch 42/100
303/303 [==============================] - 0s 279us/step - loss: 7.2555 - mae: 1.9443
Epoch 43/100
303/303 [==============================] - 0s 262us/step - loss: 7.1917 - mae: 1.9736
Epoch 44/100
303/303 [==============================] - 0s 284us/step - loss: 7.0329 - mae: 1.8979
Epoch 45/100
303/303 [==============================] - 0s 268us/step - loss: 7.0659 - mae: 1.9328
Epoch 46/100
303/303 [==============================] - 0s 265us/step - loss: 6.9414 - mae: 1.9052
Epoch 47/100
303/303 [==============================] - 0s 262us/step - loss: 6.9257 - mae: 1.8950
Epoch 48/100
303/303 [==============================] - 0s 268us/step - loss: 6.7716 - mae: 1.8816
Epoch 49/100
303/303 [==============================] - 0s 271us/step - loss: 6.8721 - mae: 1.9041
Epoch 50/100
303/303 [=

303/303 [==============================] - 0s 272us/step - loss: 9.8599 - mae: 2.1860
Epoch 23/100
303/303 [==============================] - 0s 261us/step - loss: 9.6034 - mae: 2.1716
Epoch 24/100
303/303 [==============================] - 0s 265us/step - loss: 9.4363 - mae: 2.1405
Epoch 25/100
303/303 [==============================] - 0s 266us/step - loss: 9.2344 - mae: 2.1125
Epoch 26/100
303/303 [==============================] - 0s 267us/step - loss: 9.0330 - mae: 2.0935
Epoch 27/100
303/303 [==============================] - 0s 265us/step - loss: 8.8303 - mae: 2.0997
Epoch 28/100
303/303 [==============================] - 0s 262us/step - loss: 8.8587 - mae: 2.0596
Epoch 29/100
303/303 [==============================] - 0s 257us/step - loss: 8.7544 - mae: 2.0736
Epoch 30/100
303/303 [==============================] - 0s 262us/step - loss: 8.6144 - mae: 2.0198
Epoch 31/100
303/303 [==============================] - 0s 267us/step - loss: 8.5020 - mae: 2.0527
Epoch 32/100
303/303 [=

In [9]:
all_scores

[1.9016305208206177, 2.7204136848449707, 2.6964452266693115, 2.485610008239746]

In [10]:
np.mean(all_scores)

2.4510248601436615

In [11]:
print(f'all_scores : {all_scores}')
print(f'mean all scores : {np.mean(all_scores)}')

all_scores : [1.9016305208206177, 2.7204136848449707, 2.6964452266693115, 2.485610008239746]
mean all scores : 2.4510248601436615


In [12]:
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

102/102 [==============================] - 0s 153us/step


In [13]:
test_mae_score

3.0258710384368896

In [15]:
# 아래 모델은 k fold cross validation에서 마지막에 훈련된 모델입니다.
# 본 실습에서 별도의 test data set을 구성하지 않았기 때문에, vadliation data set을 이용해 house price 를 예측prediction 해보겠습니다.

predictedHousePrice = model.predict(val_data)

In [51]:
# validation set 은 100개의 데이터 셋으로 구성되어 있습니다만 아래와 같이 1대1 비교를 해보겠습니다.
# val_data[0] 에 대한 예측값 predictedHousePrice[0]
# val_data[0] 에 대한 실제값 val_targets[0]
a = int(val_targets.shape[0])
err_sum = 0
for i in range(1, a):
    error = val_targets[i-1:i]-predictedHousePrice[i-1:i]
    #print(predictedHousePrice[i-1:i], "\n", val_targets[i-1:i])
    err_sum += error[0][0]
    #print('차이:', error[0][0])
err_avg = err_sum / a
print(err_avg)

0.6196311724067916


## 예측값과 타겟값의 차이의 평균이 0.6으로 Good